In [1]:
%autosave 0

Autosave disabled


In [11]:
import pandas as pd
import numpy as np
import xgboost as xgb
import myfunctions as mf
from sklearn.model_selection import StratifiedKFold, train_test_split

In [3]:
df = pd.read_csv('prepped_data.csv')

train_labels = pd.read_csv('../../data/prepared/train_labels.csv')
val_labels = pd.read_csv('../../data/prepared/val_labels.csv')

In [4]:
all_labels = pd.concat([train_labels, val_labels], axis=0)

In [5]:
df = df.merge(all_labels, how='left', on='customer_ID')
df.head()

,customer_ID,P_2_min,P_2_max,P_2_median,P_2_std,P_2_last,P_2_change,D_39_min,D_39_max,D_39_median,...,D_137_nulls,D_138_nulls,D_139_nulls,D_140_nulls,D_141_nulls,D_142_nulls,D_143_nulls,D_144_nulls,D_145_nulls,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580,0.960384,0.938469,0.024194,0.934745,-0.003724,0.001082,0.091505,0.002483,...,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183,0
1,0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57d...,0.068484,0.186780,0.127632,0.083648,0.068484,-0.118296,0.031711,0.914384,0.473048,...,-3.166442,-3.166442,-0.283183,-0.219693,-0.283183,-1.582977,-0.283183,-0.219331,-0.283183,1
2,0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fce...,0.837273,0.946487,0.894418,0.030962,0.875766,-0.018652,0.004118,0.740627,0.413056,...,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183,0
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...,0.365222,0.646557,0.417593,0.081273,0.410590,-0.235967,0.000303,0.213325,0.006578,...,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183,0
4,0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194f...,0.752942,0.867515,0.843999,0.030110,0.850308,0.097366,0.000355,0.009881,0.007834,...,0.446319,0.446319,1.033397,-0.219693,1.033397,0.596403,1.033397,-0.219331,1.033397,0


In [6]:
seed = 42

train, val = train_test_split(df, train_size=0.9,
                              random_state=seed)

In [7]:
train_X = train.drop(columns=['customer_ID', 'target'])
train_y = train['target']

val_X = val.drop(columns=['customer_ID', 'target'])
val_y = val['target']

In [8]:
train_matrix = xgb.DMatrix(train_X, label=train_y)
val_matrix = xgb.DMatrix(val_X, label=val_y)

In [9]:
params = {
    'max_depth': 4,
    'learning_rate': 0.1,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'min_child_weight': 10,
    'objective': 'binary:logistic',
    'random_state': seed,
    'verbosity': 1
}

In [10]:
model = xgb.train(params, train_matrix, num_boost_round=2_000,
                  evals=[(train_matrix, 'train'), (val_matrix, 'validate')],
                  early_stopping_rounds=32)

[0]	train-logloss:0.63262	validate-logloss:0.63276
[1]	train-logloss:0.58305	validate-logloss:0.58331
[2]	train-logloss:0.54162	validate-logloss:0.54191
[3]	train-logloss:0.50630	validate-logloss:0.50658
[4]	train-logloss:0.47630	validate-logloss:0.47672
[5]	train-logloss:0.45026	validate-logloss:0.45063
[6]	train-logloss:0.42804	validate-logloss:0.42851
[7]	train-logloss:0.40835	validate-logloss:0.40891
[8]	train-logloss:0.39127	validate-logloss:0.39180
[9]	train-logloss:0.37608	validate-logloss:0.37656
[10]	train-logloss:0.36271	validate-logloss:0.36318
[11]	train-logloss:0.35090	validate-logloss:0.35134
[12]	train-logloss:0.34043	validate-logloss:0.34098
[13]	train-logloss:0.33110	validate-logloss:0.33166
[14]	train-logloss:0.32278	validate-logloss:0.32331
[15]	train-logloss:0.31535	validate-logloss:0.31590
[16]	train-logloss:0.30867	validate-logloss:0.30927
[17]	train-logloss:0.30260	validate-logloss:0.30322
[18]	train-logloss:0.29691	validate-logloss:0.29763
[19]	train-logloss:0.2

[157]	train-logloss:0.21285	validate-logloss:0.22398
[158]	train-logloss:0.21274	validate-logloss:0.22399
[159]	train-logloss:0.21262	validate-logloss:0.22389
[160]	train-logloss:0.21247	validate-logloss:0.22384
[161]	train-logloss:0.21236	validate-logloss:0.22381
[162]	train-logloss:0.21224	validate-logloss:0.22379
[163]	train-logloss:0.21216	validate-logloss:0.22374
[164]	train-logloss:0.21208	validate-logloss:0.22374
[165]	train-logloss:0.21202	validate-logloss:0.22372
[166]	train-logloss:0.21189	validate-logloss:0.22370
[167]	train-logloss:0.21178	validate-logloss:0.22364
[168]	train-logloss:0.21164	validate-logloss:0.22361
[169]	train-logloss:0.21151	validate-logloss:0.22354
[170]	train-logloss:0.21140	validate-logloss:0.22350
[171]	train-logloss:0.21131	validate-logloss:0.22348
[172]	train-logloss:0.21119	validate-logloss:0.22348
[173]	train-logloss:0.21105	validate-logloss:0.22345
[174]	train-logloss:0.21098	validate-logloss:0.22347
[175]	train-logloss:0.21089	validate-logloss:0

[312]	train-logloss:0.19983	validate-logloss:0.22226
[313]	train-logloss:0.19974	validate-logloss:0.22227


In [23]:
result = mf.model_evaluator(model, val_matrix, val_y)

result

-0.009403183324189837

In [16]:
y_hat = model.predict(val_matrix)

In [17]:
y_true = pd.DataFrame(val_y)

In [18]:
y_hat_final = pd.DataFrame(y_hat, columns=['prediction'])

In [19]:
mf.amex_metric(y_true, y_hat_final)

-0.009403183324189837

In [20]:
model.predict(val_matrix)

array([7.5119799e-01, 2.9902876e-04, 4.7911796e-01, ..., 7.2137153e-01,
       1.1441959e-03, 5.5878204e-03], dtype=float32)

In [21]:
val_y

203562    1
206859    0
68697     1
175493    0
5922      0
         ..
142352    0
3564      0
51647     1
39864     0
202799    0
Name: target, Length: 25316, dtype: int64

Code for cross-validation

In [9]:
matrix = xgb.DMatrix(df.drop(columns=['customer_ID', 'target']), label=df['target'])

In [14]:
seed = 42

skf = StratifiedKFold(n_splits=5)

params = {
    'max_depth': 4,
    'learning_rate': 0.1,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'min_child_weight': 10,
    'objective': 'binary:logistic',
    'random_state': seed
}

In [15]:
xgb.cv(params, matrix, num_boost_round=2_000, nfold=5,
       folds=skf, metrics='logloss', early_stopping_rounds=32,
       verbose_eval=True, seed=seed)

[0]	train-logloss:0.63257+0.00007	test-logloss:0.63264+0.00011
[1]	train-logloss:0.58314+0.00050	test-logloss:0.58337+0.00059


KeyboardInterrupt: 